In [88]:
import pandas as pd
import numpy as np
import os
import bs4
import requests
from time import sleep
from lib import utils
import json
from IPython.display import display, clear_output

In [102]:
file_list = os.listdir('character_pages/')

In [103]:
content_dict = {}
categories_dict = {}
bio_dict = {}

for counter, file in enumerate(file_list, start=1):
    with open('character_pages/' + file, 'r') as obj:
        page_html = obj.read()
    
    char = file.split('.')[0]
    page_soup = bs4.BeautifulSoup(page_html)
    
    clear_output()
    print('Parsing file {} of {}: {}'.format(counter, len(file_list), file), end='\r')
    
    # Content
    content_elms = page_soup.select('div .mw-content-ltr > p')
    content_txt = ' '.join([content_elms[x].getText(separator=' ',strip=True) for x in range(len(content_elms))])
    content_dict[char] = content_txt
    
    # Character Categories
    cate = page_soup.select('div .mw-normal-catlinks > ul')[0].getText(separator=',', strip=True)
    categories_dict[char] = cate
    
    # Character biography
    bio_elms = page_soup.select('div .mw-content-ltr tr[style="vertical-align: top; text-align: left;"]')
    bio_dict[char] = {}
    for elm in bio_elms:
        key = elm.select('td')[0].getText().strip()
        if key == 'Books':
            value = elm.select('td')[1].getText(separator=',', strip=True)
        else:
            value = elm.select('td')[1].getText(separator=' ', strip=True)
        bio_dict[char][key] = value

In [104]:
content_df = pd.DataFrame({
    'char': list(content_dict.keys())
    , 'content': list(content_dict.values())
})

In [111]:
content_df.to_csv('data/raw/character_contents.csv', index=False)

In [105]:
categories_df = pd.DataFrame({
    'char': list(categories_dict.keys())
    , 'categories': list(categories_dict.values())
})

In [112]:
categories_df.to_csv('data/raw/character_categories.csv', index=False)

In [106]:
bio_df = pd.DataFrame({
    'char': list(bio_dict.keys())
})

In [107]:
cols = ['Name', 'Race', 'Age', 'Occupation', 'Physical appearance'
        , 'Residence', 'Death', 'Parents', 'Relatives', 'Children'
        , 'Marital Status', 'Books', 'Cameos']

In [108]:
for counter, char in enumerate(bio_dict.keys()):
    for col in cols:
        if col not in bio_dict[char].keys():
            bio_df.loc[counter, col] = np.nan
        else:
            bio_df.loc[counter, col] = bio_dict[char][col]

In [113]:
bio_df.to_csv('data/raw/character_bios.csv', index=False)